# Climatology Computation

The user can specify:
- Path to ECMWF data (see structure of folders and files in the default path)
- Output directory: where all the outputs will be saved
- Start Date and End Date of Climatology Computation

In [1]:
import os
import glob
from datetime import datetime
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

In [2]:
# Define the data directory and the variables of interest
data_dir = '/data/inputs/METOCEAN/historical/model/atmos/ECMWF/IFS_010/analysis/6h/netcdf'
output_dir = '/work/cmcc/mg28621/out_ECMWF_Tool_Checks/'
os.makedirs(output_dir, exist_ok=True)
variables = ['T2M', 'D2M', 'MSL', 'U10M', 'V10M', 'LSM']

# Define the date range
start_date = '2019-01-01'
end_date = '2023-12-31'

In [3]:
# Convert the date strings to datetime objects
start_date_dt = datetime.strptime(start_date, '%Y-%m-%d')
end_date_dt = datetime.strptime(end_date, '%Y-%m-%d')

# Extract the year from the datetime objects
start_year = start_date_dt.year
end_year = end_date_dt.year

# Create a list to store data for each variable
data = {var: [] for var in variables}
time_index = []

# For the moment being these dates need to be excluded from from climatology calculation for all variables, due to Land Sea Mask Errors
# If you decide to change the period and you are sure no issues exist, you can modify this line in: excluded_dates = pd.to_datetime([])
excluded_dates = pd.to_datetime([
    '2023-12-03 12:00', 
    '2023-11-01 06:00'
])

In [ ]:
# Loop through each year and month to find the corresponding files
for year in range(2019, 2024):
    print(year)
    for month in range(1, 13):
        print(month)
        if f'{year}-{month:02d}' > end_date[:7]:
            break
        if f'{year}-{month:02d}' < start_date[:7]:
            continue
        # Create the file pattern for the current month
        file_pattern = os.path.join(data_dir, f'{year}/{month:02d}', '*MEDATL*.nc')
        files = sorted(glob.glob(file_pattern))
        
        # Process each file
        for file in files:
            # Open the dataset
            ds = xr.open_dataset(file)
            
            # Extract the time values
            times = pd.to_datetime(ds['time'].values)
            
            # Filter out the excluded dates for all variables
            valid_times = ~times.isin(excluded_dates)
            times = times[valid_times]
            
            # Store data for valid times
            for var in variables:
                data[var].append(ds[var].values[valid_times])
            time_index.extend(times)
            
            # Close the dataset
            ds.close()

In [5]:
# Combine data into xarray datasets
data_arrays = {var: xr.DataArray(np.concatenate(data[var], axis=0), coords=[time_index, ds['lat'].values, ds['lon'].values], dims=['time', 'lat', 'lon']) for var in variables}
# Apply Land Sea Mask on variables
for var in variables:
    if var != 'LSM':
        print(var)
        data_arrays[var] = data_arrays[var] * np.logical_not(data_arrays['LSM'])
        data_arrays[var] = data_arrays[var].where(np.logical_not(data_arrays['LSM']) !=0, np.nan)

In [6]:
# For the moment being the bissextile years are considered as non-bissextile years

time_da = xr.DataArray(time_index, dims='time')

is_leap_year = time_da.dt.is_leap_year
is_feb_29 = (time_da.dt.month == 2) & (time_da.dt.day == 29)
valid_days = ~(is_leap_year & is_feb_29)

valid_days = valid_days.values

time_index_filtered = np.array(time_index)[valid_days]

time_da_filtered = xr.DataArray(time_index_filtered, dims='time')

leap_years = np.unique(time_da_filtered.dt.year[time_da_filtered.dt.is_leap_year])
is_after_feb_29_leap_year = np.zeros(time_da_filtered.size, dtype=bool)
for year in leap_years:
    # Identify dates after February 29th for the current leap year
    is_current_leap_year = (time_da_filtered.dt.year == year)
    is_after_feb_29 = (time_da_filtered.dt.month > 2) | ((time_da_filtered.dt.month == 2) & (time_da_filtered.dt.day > 29))
    # Combine conditions
    is_after_feb_29_leap_year |= (is_current_leap_year & is_after_feb_29)

time_da_corrected = time_da_filtered.copy()
time_da_corrected.values[is_after_feb_29_leap_year] -= np.timedelta64(1, 'D')

dayofyear = time_da_corrected.dt.dayofyear

In [7]:
# Define Med Sea limits
lat_min, lat_max = 30, 46
lon_min, lon_max = -6, 40.2

# Filtering data only for Med Sea
mediterranean_data_arrays = {
    var: array.sel(lat=slice(lat_max, lat_min), lon=slice(lon_min, lon_max)) for var, array in data_arrays.items()
}

# Mask for Med Sea, excluded Atlantic ad in-land Lakes
lat = mediterranean_data_arrays['LSM'].lat
lon = mediterranean_data_arrays['LSM'].lon

mask_atlantic = (lat >42 ) & (lon > -6) & (lon < 0)
mask_inland_lakes = (lat > 37) & (lat < 40) & (lon > 27.8)

total_mask = mask_atlantic | mask_inland_lakes

# Apply mask to data
for var in mediterranean_data_arrays:
    mediterranean_data_arrays[var] = mediterranean_data_arrays[var].where(~total_mask)

In [46]:
# Plot of LSM sum time series
filtered_lsm = mediterranean_data_arrays['LSM'].where(mediterranean_data_arrays['LSM'] != 1, np.nan)
lsm_sum = filtered_lsm.where(filtered_lsm != False, 1).sum(dim=['lat', 'lon'])

# Crea un plot della serie temporale
plt.figure(figsize=(10, 5))
lsm_sum.plot()
plt.title('LSM sum time series')
plt.xlabel('Time')
plt.ylabel('LSM elements sum')
plt.grid(True)
plt.savefig(os.path.join(output_dir, f'mediterranean_LSM_sum_time_series_{start_year}_{end_year}.png'))
plt.close()

In [ ]:
# Step 8: Group data by corrected dayofyear
climatology_stats_mediterranean = {}
for var, array in mediterranean_data_arrays.items():
    daily = array.sel(time=time_da_filtered).groupby(dayofyear)
    climatology_stats_mediterranean[var] = {
        'mean': daily.mean(dim='time',skipna=True),
        'std': daily.std(dim='time',skipna=True)
#        'min': daily.min(dim='time',skipna=True), other method to compute clim_min and max as mean of annual maxs and mins
#        'max': daily.max(dim='time',skipna=True) 
    }

    # 1. Group data by year
    yearly_grouped = array.sel(time=time_da_corrected).groupby('time.year')
    
    # 2. Inside each year, group by dayofyear and compute max for each day
    yearly_max = yearly_grouped.map(lambda x: x.groupby('time.dayofyear').max(dim='time', skipna=True))
    
    # 3. Compute clim_max for each dayofyear
    #climatology_stats_mediterranean[var]['max'] = yearly_max.mean(dim='year', skipna=True) # other method
    climatology_stats_mediterranean[var]['max'] = yearly_max.max(dim='year', skipna=True)
    
    # Same for minimum
    yearly_grouped = array.sel(time=time_da_corrected).groupby('time.year')
    yearly_min = yearly_grouped.map(lambda x: x.groupby('time.dayofyear').min(dim='time', skipna=True))

    #climatology_stats_mediterranean[var]['min'] = yearly_min.mean(dim='year', skipna=True) # altro metodo
    climatology_stats_mediterranean[var]['min'] = yearly_min.min(dim='year', skipna=True)

In [10]:
# Save the climatology statistics to NetCDF files
for var, stats in climatology_stats_mediterranean.items():
    ds_climatology_mediterranean = xr.Dataset({
        'mean': stats['mean'],
        'std': stats['std'],
        'min': stats['min'],
        'max': stats['max']
    })
    ds_climatology_mediterranean.to_netcdf(os.path.join(output_dir, f'{var}_climatology_mediterranean.nc'))

In [11]:
def plot_statistics_mediterranean(start_date_dt, end_date_dt, stat_name, stat_data, output_file):
    units = {
        'T2M': 'K',
        'D2M': 'K',
        'MSL': 'Pa',
        'U10M': 'm/s',
        'V10M': 'm/s'
    }
    fig, axes = plt.subplots(len(units.keys()), 1, figsize=(15, 20), sharex=True)
    fig.suptitle(f'Spatially Averaged Climatological Daily {stat_name.capitalize()} over the Mediterranean frim {start_date_dt.strftime("%Y-%m.%d")} to {end_date_dt.strftime("%Y-%m-%d")}', fontsize=20)
    for i, var in enumerate(units.keys()):
        day_range = pd.date_range(start=start_date_dt.replace(day=1).strftime('%Y-%m-%d'), periods=365)
        axes[i].plot(day_range, stat_data[var], label=f'{var} {stat_name.capitalize()}', color='blue')
        axes[i].set_title(f'{var} {stat_name.capitalize()}',fontsize=20)
        axes[i].set_ylabel(f'{var} ({units[var]})',fontsize=20)
        axes[i].legend(prop={'size': 6})
        axes[i].xaxis.set_major_formatter(mdates.DateFormatter('%b'))
        axes[i].xaxis.set_major_locator(mdates.MonthLocator())

    plt.xlabel('Day of Year')
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])

    # Save Plots
    plt.savefig(output_file)
    plt.close(fig)
  
# Run plots
plot_statistics_mediterranean(start_date_dt, end_date_dt, 'mean', {var: climatology_stats_mediterranean[var]['mean'].mean(dim=['lat', 'lon']) for var in variables if var != 'LSM'}, os.path.join(output_dir, f'mediterranean_climatological_daily_mean_{start_year}_{end_year}.png'))
plot_statistics_mediterranean(start_date_dt, end_date_dt, 'std', {var: climatology_stats_mediterranean[var]['std'].mean(dim=['lat', 'lon']) for var in variables if var != 'LSM'}, os.path.join(output_dir, f'mediterranean_climatological_daily_std_{start_year}_{end_year}.png'))
plot_statistics_mediterranean(start_date_dt, end_date_dt, 'min', {var: climatology_stats_mediterranean[var]['min'].mean(dim=['lat', 'lon']) for var in variables if var != 'LSM'}, os.path.join(output_dir, f'mediterranean_climatological_daily_min_{start_year}_{end_year}.png'))
plot_statistics_mediterranean(start_date_dt, end_date_dt, 'max', {var: climatology_stats_mediterranean[var]['max'].mean(dim=['lat', 'lon']) for var in variables if var != 'LSM'}, os.path.join(output_dir, f'mediterranean_climatological_daily_max_{start_year}_{end_year}.png'))

In [14]:
def plot_statistics_mediterranean_with_std(start_date_dt, end_date_dt, stat_name, stat_mean_data, stat_std_data, output_file):
    units = {
        'T2M': 'K',
        'D2M': 'K',
        'MSL': 'Pa',
        'U10M': 'm/s',
        'V10M': 'm/s'
    }
    fig, axes = plt.subplots(len(units.keys()), 1, figsize=(15, 20), sharex=True)
    fig.suptitle(f'Spatially Averaged Climatological Daily {stat_name.capitalize()} over the Mediterranean from {start_date_dt.strftime("%Y-%m.%d")} to {end_date_dt.strftime("%Y-%m.%d")}', fontsize=19)

    for i, var in enumerate(units.keys()):
        day_range = pd.date_range(start=start_date_dt.replace(day=1).strftime('%Y-%m-%d'), periods=365)
        
        # Daily Climatology
        axes[i].plot(day_range, stat_mean_data[var], label=f'{var} Mean', color='blue')
        
        # Daily Climatology + std, 2*std, 3*std
        axes[i].plot(day_range, stat_mean_data[var] + stat_std_data[var], label=f'{var} Mean + 1*std', color='orange', linestyle='--')
        axes[i].plot(day_range, stat_mean_data[var] + 2 * stat_std_data[var], label=f'{var} Mean + 2*std', color='green', linestyle='--')
        axes[i].plot(day_range, stat_mean_data[var] + 3 * stat_std_data[var], label=f'{var} Mean + 3*std', color='red', linestyle='--')
        
        axes[i].set_title(f'{var} {stat_name.capitalize()} with Standard Deviations',fontsize=20)
        axes[i].set_ylabel(f'{var} ({units[var]})',fontsize=20)
        axes[i].legend(prop={'size': 6})
        axes[i].xaxis.set_major_formatter(mdates.DateFormatter('%b'))
        axes[i].xaxis.set_major_locator(mdates.MonthLocator())

    plt.xlabel('Day of Year')
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])

    # Save Plots
    plt.savefig(output_file)
    plt.close(fig)

# Run plots
plot_statistics_mediterranean_with_std(start_date_dt, end_date_dt,
    'mean + std',
    {var: climatology_stats_mediterranean[var]['mean'].mean(dim=['lat', 'lon']) for var in variables if var != 'LSM'},
    {var: climatology_stats_mediterranean[var]['std'].mean(dim=['lat', 'lon']) for var in variables if var != 'LSM'},
    os.path.join(output_dir, f'mediterranean_climatological_daily_mean_with_std_{start_year}_{end_year}.png')
)

# Checks against climatology

The user can set:
- start_date, end_date of the period in which he/she wants to check the data
- data_dir: directory to the ecmwft .nc data
- clim_dir: directory containing the .nc climatology data
- output_check_dir: directory of these checks output (can be also the same of output dir of climatology computation)

In [3]:
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.colors import TwoSlopeNorm
from matplotlib.ticker import FormatStrFormatter
import datetime
from datetime import datetime
import pandas as pd
import xarray as xr
import netCDF4 as nc
import glob

In [4]:
def check_range(value, climatology, dayofyear):
    """Check if the value is within the climatological min and max range, handling np.nan correctly."""
    #clim_min = climatology['min'].sel(dayofyear=dayofyear)
    clim_mean = climatology['mean'].sel(dayofyear=dayofyear)
    #clim_max = climatology['max'].sel(dayofyear=dayofyear)
    clim_std = climatology['std'].sel(dayofyear=dayofyear)

    #nan_mask = np.isnan(value) | np.isnan(clim_min) | np.isnan(clim_max
    nan_mask = np.isnan(value) | np.isnan(clim_mean) | np.isnan(clim_std)
    #within_range = (value >= clim_min) & (value <= clim_max)
    within_range = (value >= clim_mean - 2*clim_std) & (value <= clim_mean + 2*clim_std)

    # Apply the NaN mask: where NaN is present, the result should be NaN
    within_range = within_range.where(~nan_mask, np.nan)
   # Invert the boolean array (transforming 1 into 0, and 0 into 1)
    out_of_range = xr.where(within_range == False, 1, 0)

    # Ensure NaN values remain unchanged
    out_of_range = out_of_range.where(~nan_mask, np.nan)

    return out_of_range

def check_max(value, climatology, dayofyear):
    """Check if the value is less than or equal to the climatological max, handling np.nan correctly."""
    clim_max = climatology['max'].sel(dayofyear=dayofyear)

    nan_mask = np.isnan(value) | np.isnan(clim_max)
    within_range = np.abs(value) <= np.abs(clim_max)
    
    # Apply the NaN mask: where NaN is present, the result should be NaN
    within_range = within_range.where(~nan_mask, np.nan)
   # Invert the boolean array (transforming 1 into 0, and 0 into 1)
    out_of_range = xr.where(within_range == False, 1, 0)
    
    # Ensure NaN values remain unchanged
    out_of_range = out_of_range.where(~nan_mask, np.nan)

    return out_of_range

def check_min(value, climatology, dayofyear):
    """Check if the value is greater than or equal to the climatological min, handling np.nan correctly."""
    clim_min = climatology['min'].sel(dayofyear=dayofyear)
    
    nan_mask = np.isnan(value) | np.isnan(clim_min)
    within_range = np.abs(value) <= np.abs(clim_min)
    
    # Apply the NaN mask: where NaN is present, the result should be NaN
    within_range = within_range.where(~nan_mask, np.nan)
   # Invert the boolean array (transforming 1 into 0, and 0 into 1)
    out_of_range = xr.where(within_range == False, 1, 0)
    
    # Ensure NaN values remain unchanged
    out_of_range = out_of_range.where(~nan_mask, np.nan)

    return out_of_range

def load_climatology_stats(climatology_dir):
    """
    Load the precomputed climatological statistics.
    Here we assume these stats are stored in a format similar to:
    climatology_stats['T2M'] = {'mean': ..., 'min': ..., 'max': ..., 'std': ...}
    """
    climatology_stats = {}
    variables = ['T2M', 'D2M', 'MSL', 'U10M', 'V10M', 'LSM']
    
    for var in variables:
        ds = xr.open_dataset(os.path.join(climatology_dir, f'{var}_climatology_mediterranean.nc'))
        climatology_stats[var] = {
            'mean': ds['mean'],
            'min': ds['min'],
            'max': ds['max'],
            'std': ds['std']
        }
    
    return climatology_stats

def align_to_climatology(ds, climatology_stats):
    """
    Align the spatial domain of 'ds' to the lat/lon range of the climatology_stats.
    
    Parameters:
    - ds: xarray.Dataset, the dataset to align
    - climatology_stats: dict, the precomputed climatology statistics

    Returns:
    - ds_aligned: xarray.Dataset, the dataset aligned to the climatology spatial domain
    """
    # Extract the lat/lon ranges from the climatology LSM mask
    lat_range = climatology_stats['LSM']['mean']['lat']
    lon_range = climatology_stats['LSM']['mean']['lon']
    
    # Slice ds to match the climatology lat/lon ranges
    ds_aligned = ds.sel(lat=slice(lat_range.max(), lat_range.min()), lon=slice(lon_range.min(), lon_range.max()))
    ds_aligned = ds_aligned.where(~np.isnan(climatology_stats['LSM']['mean'][0]), np.nan)
    
    return ds_aligned

def extract_anomalies(ds, alerts, date, output_plot_dir):
    """Extract anomalies based on alerts for a specific date."""
    anomalies = {}
    
    var_mapping = {
        'T2M_mean_alert': 'T2M',
        'D2M_mean_alert': 'D2M',
        'MSL_mean_alert': 'MSL',
        'U10M_mean_alert': 'U10M',
        'V10M_mean_alert': 'V10M',
        'U10M_max_alert': 'U10M',
        'U10M_min_alert': 'U10M',
        'V10M_max_alert': 'V10M',
        'V10M_min_alert': 'V10M'
    }

    latitudes = ds['lat'].values
    longitudes = ds['lon'].values
    
    # Create subfolder for the date
    day_dir = os.path.join(output_plot_dir, date.strftime('%Y-%m-%d'))
    os.makedirs(day_dir, exist_ok=True)
    
    for var, alert in alerts.items():
        if isinstance(alert, xr.DataArray):
            if var in var_mapping:
                data_var = var_mapping[var]
                print("plot alert map")
                plt.figure(figsize=(12, 8))
                alert.plot()
                print("save")
                plt.savefig(f"{day_dir}/alert_map_{var}_{date.strftime('%Y-%m-%d')}")
                plt.close()
            else:
                continue
            alert_values = alert.values
            # Find alert indices inside the domain
            alert_indices = np.argwhere(alert_values == 1)
            
            if len(alert_indices) > 0:
                lat_alerts = latitudes[alert_indices[:, 0]]
                lon_alerts = longitudes[alert_indices[:, 1]]
                
                values_alerts = np.array([
                    ds[data_var].sel(lat=latitudes[lat_idx], lon=longitudes[lon_idx]).values
                    for lat_idx, lon_idx in alert_indices
                ])
                
                anomalies[var] = {
                    'latitude': lat_alerts,
                    'longitude': lon_alerts,
                    'values': values_alerts
                }
    
    return anomalies

def generate_alert_plots(date, ds, climatology_stats, alerts, dayofyear, output_plot_dir):
    """Generate and save plots for variables with alert and create CSV files."""
    variable_map = {
        'T2M_mean_alert': 'T2M',
        'D2M_mean_alert': 'D2M',
        'MSL_mean_alert': 'MSL',
        'U10M_mean_alert': 'U10M',
        'V10M_mean_alert': 'V10M',
        'U10M_max_alert': 'U10M',
        'V10M_max_alert': 'V10M',
        'U10M_min_alert': 'U10M',
        'V10M_min_alert': 'V10M',
    }

    anomalies = extract_anomalies(ds, alerts, date, output_plot_dir)
    
    day_dir = os.path.join(output_plot_dir, date.strftime('%Y-%m-%d'))
    os.makedirs(day_dir, exist_ok=True)
    
    for alert_var, anomaly_data in anomalies.items():
        var = variable_map.get(alert_var)
        if var is None:
            print(f"No mapping found for alert variable: {alert_var}")
            continue
        
        # Create CSV file
        csv_data = []
        csv_data_uv = []
        for lat, lon, value in zip(anomaly_data['latitude'], anomaly_data['longitude'], anomaly_data['values']):
            if var in ['T2M', 'D2M', 'MSL','U10M','V10M']:
                # Compute mean of values_alerts
                value_mean = value.mean()
                clim_max = climatology_stats[var]['max'].sel(dayofyear=dayofyear, lat=lat, lon=lon, method='nearest').values
                clim_mean = climatology_stats[var]['mean'].sel(dayofyear=dayofyear, lat=lat, lon=lon, method='nearest').values
                clim_min = climatology_stats[var]['min'].sel(dayofyear=dayofyear, lat=lat, lon=lon, method='nearest').values
                
                csv_data.append([lat, lon, value, value_mean, clim_min, clim_mean, clim_max])
            
            elif var in ['U10M', 'V10M']:
                # Compute max and min of values_alerts
                value_max = value.max()
                value_min = value.min()
                clim_max = climatology_stats[var]['max'].sel(dayofyear=dayofyear, lat=lat, lon=lon, method='nearest').values
                clim_min = climatology_stats[var]['min'].sel(dayofyear=dayofyear, lat=lat, lon=lon, method='nearest').values
                
                csv_data_uv.append([lat, lon, value, value_max, clim_max, value_min, clim_min])
        
        if var in ['T2M', 'D2M', 'MSL','U10M','V10M']:
            csv_columns = ['lat_alerts', 'lon_alerts', 'values_alerts', 'mean_value', 'climatology_min', 'climatology_mean' ,'climatology_max']
        elif var in ['U10M', 'V10M']:
            csv_columns = ['lat_alerts', 'lon_alerts', 'values_alerts', 'max_value', 'climatology_max', 'min_value', 'climatology_min']
        
        # Save CSV files
        csv_path = os.path.join(day_dir, f'{var}_anomalies_{date.strftime("%Y%m%d")}.csv')
        df = pd.DataFrame(csv_data, columns=csv_columns)
        df.to_csv(csv_path, index=False)

        csv_uv_path = os.path.join(day_dir, f'{var}_anomalies_{date.strftime("%Y%m%d")}.csv')
        df = pd.DataFrame(csv_data_uv, columns=csv_columns)
        df.to_csv(csv_uv_path, index=False)       

def calculate_time_series(climatology_stats, start_date, end_date, date_range, data_dir):
    """
    Compute time series for variables T2M, D2M, MSL, U10M e V10M and for the corresponding climatological time series
    """
    
    time_series = {
        'T2M': [],
        'D2M': [],
        'MSL': [],
        'LSM': [],
        'U10M': [],
        'V10M': [],
        'U10M_max': [],
        'U10M_min': [],
        'V10M_max': [],
        'V10M_min': []
    }
    
    clim_series = {
        'T2M': {'mean': [], 'max': [], 'min': [], 'std': []},
        'D2M': {'mean': [], 'max': [], 'min': [], 'std': []},
        'MSL': {'mean': [], 'max': [], 'min': [], 'std': []},
        'U10M': {'mean': [], 'max': [], 'min': [], 'std': []},
        'V10M': {'mean': [], 'max': [], 'min': [], 'std': []}
    }
    
    for date in date_range:
        print(date)
        year, month, day = date.year, date.month, date.day
        file_pattern = os.path.join(data_dir, f'{year}/{month:02d}', f'*{year:02d}{month:02d}{day:02d}*MEDATL*.nc')
        files = sorted(glob.glob(file_pattern))
        
        if not files:
            print(f"No files found for {date}")
            continue
        
        for file in files:
            print(file)
            print("apri file")
            ds = xr.open_dataset(file)
            
        # Align ds to the climatology domain
        ds_aligned = align_to_climatology(ds, climatology_stats)
        ds_day = ds_aligned
        
        daily_mean = ds_day[['T2M', 'D2M', 'MSL','U10M','V10M']].mean(dim=['lat', 'lon'])
        daily_max = ds_day[['U10M', 'V10M']].max(dim=['time']).mean(dim=['lat', 'lon'])
        daily_min = ds_day[['U10M', 'V10M']].min(dim=['time']).mean(dim=['lat', 'lon'])
        daily_sum = ds_day[['LSM']].where(ds_day['LSM'] != False, 1).sum(dim=['lat', 'lon'])
        
        time_series['T2M'].append(daily_mean['T2M'].values)
        time_series['D2M'].append(daily_mean['D2M'].values)
        time_series['MSL'].append(daily_mean['MSL'].values)
        time_series['LSM'].append(daily_sum['LSM'].values)
        time_series['U10M'].append(daily_mean['U10M'].values)
        time_series['V10M'].append(daily_mean['V10M'].values)
        time_series['U10M_max'].append(daily_max['U10M'].values)
        time_series['U10M_min'].append(daily_min['U10M'].values)
        time_series['V10M_max'].append(daily_max['V10M'].values)
        time_series['V10M_min'].append(daily_min['V10M'].values)
    
    for var in ['T2M', 'D2M', 'MSL', 'U10M', 'V10M']:
        clim_mean = climatology_stats[var]['mean'].mean(dim=['lat', 'lon'])
        clim_max = climatology_stats[var]['max'].mean(dim=['lat', 'lon'])
        clim_min = climatology_stats[var]['min'].mean(dim=['lat', 'lon'])
        clim_std = climatology_stats[var]['std'].mean(dim=['lat', 'lon'])
        # Repeat climatological series for the number of years in the range if needed

        if (datetime.strptime(end_date, '%Y-%m-%d') - datetime.strptime(start_date, '%Y-%m-%d')).days > 365:
            years = (datetime.strptime(end_date, '%Y-%m-%d') - datetime.strptime(start_date, '%Y-%m-%d')).days // 365 + 1
            clim_mean = clim_mean.expand_dims(dim='time').broadcast_like(clim_mean, exclude={dim: years for dim in clim_mean.dims})
            clim_max = clim_max.expand_dims(dim='time').broadcast_like(clim_max, exclude={dim: years for dim in clim_max.dims})
            clim_min = clim_min.expand_dims(dim='time').broadcast_like(clim_min, exclude={dim: years for dim in clim_min.dims})
            clim_std = clim_std.expand_dims(dim='time').broadcast_like(clim_std, exclude={dim: years for dim in clim_std.dims})

        clim_series[var]['mean'] = clim_mean
        clim_series[var]['max'] = clim_max
        clim_series[var]['min'] = clim_min
        clim_series[var]['std'] = clim_std
    
    for var in ['U10M', 'V10M']:
        clim_max = climatology_stats[var]['max'].mean(dim=['lat', 'lon'])
        clim_min = climatology_stats[var]['min'].mean(dim=['lat', 'lon'])
        # Repeat climatological series for the number of years in the range if needed
        if (datetime.strptime(end_date, '%Y-%m-%d') - datetime.strptime(start_date, '%Y-%m-%d')).days > 365:
            years = (datetime.strptime(end_date, '%Y-%m-%d') - datetime.strptime(start_date, '%Y-%m-%d')).days // 365 + 1
            clim_max = clim_max.expand_dims(dim='time').broadcast_like(clim_max, {dim: years for dim in clim_max.dims})
            clim_min = clim_min.expand_dims(dim='time').broadcast_like(clim_min, {dim: years for dim in clim_min.dims})
        
        clim_series[var]['max'] = clim_max
        clim_series[var]['min'] = clim_min

    
    return time_series, clim_series, ds_day

def plot_time_series(time_series, clim_series, start_date, end_date, date_range, date_range_4h, output_dir):
    """
    Create and save plots of time series for specified variable.
    """
    variable_map = {
        'U10M_max': 'U10M',
        'V10M_max': 'V10M',
        'U10M_min': 'U10M',
        'V10M_min': 'V10M',
    }
    
    start_year = datetime.strptime(start_date, '%Y-%m-%d').year
    end_year = datetime.strptime(end_date, '%Y-%m-%d').year
    
    # Creiamo una data range per l'anno climatologico di riferimento (es. 2020)
    climatology_dates = pd.date_range(start=f'{start_year}-01-01', end=f'{start_year}-12-31', freq='D')
    # Rimuovere il 29 febbraio se esiste
    climatology_dates = climatology_dates[~((climatology_dates.month == 2) & (climatology_dates.day == 29))]    
    
    for var in time_series:
        os.makedirs(os.path.join(output_dir, f'{var}'), exist_ok=True)
        print(os.path.join(output_dir, f'{var}'))
        if var in ['T2M', 'D2M', 'MSL', 'U10M', 'V10M']:
            
            # Se l'intervallo copre più di un anno, ripetiamo la serie climatologica per tutti gli anni necessari
            if end_year > start_year:
                extended_dates = pd.date_range(start=f'{start_year}-01-01', end=f'{end_year}-12-31', freq='D')
                extended_dates = extended_dates[~((extended_dates.month == 2) & (extended_dates.day == 29))]  # Rimuovere eventuali 29 febbraio
                climatology_values_mean = np.tile(clim_series[var]['mean'].values, len(extended_dates) // 365)[0]
                climatology_values_std = np.tile(clim_series[var]['std'].values, len(extended_dates) // 365)[0]
                climatology_values_max = np.tile(clim_series[var]['max'].values, len(extended_dates) // 365)[0]
                climatology_values_min = np.tile(clim_series[var]['min'].values, len(extended_dates) // 365)[0]
            else:
                extended_dates = climatology_dates
                climatology_values_std = clim_series[var]['std'].values
                climatology_values_mean = clim_series[var]['mean'].values
                climatology_values_max = clim_series[var]['max'].values
                climatology_values_min = clim_series[var]['min'].values
                
            plt.figure(figsize=(12, 8))
            plt.plot(date_range_4h, np.concatenate(time_series[var]), label=f'{var} observed', color='blue')
            plt.plot(extended_dates, climatology_values_mean, label=f'{var} climatological mean', color='red', linestyle='--')
            plt.plot(extended_dates, climatology_values_max, label=f'{var} climatological max', color='green', linestyle='--')
            plt.plot(extended_dates, climatology_values_min, label=f'{var} climatological min', color='orange', linestyle='--')
            
            plt.xlabel('Date')
            plt.ylabel(f'{var}')
            plt.title(f'{var} Time Series Comparison With Climatology')
            plt.legend()
            plt.grid(True)
            
            # Save the plot
            plot_path = os.path.join(output_dir, f'{var}/{var}_time_series_{start_date}_{end_date}.png')
            plt.savefig(plot_path)
            plt.close()

            plt.figure(figsize=(12, 8))
            plt.plot(date_range, np.array([np.mean(arr) for arr in time_series[var]]), label=f'{var} observed', color='blue')
            plt.plot(extended_dates, climatology_values_mean, label=f'{var} climatological mean', color='red', linestyle='--')
            plt.fill_between(extended_dates, climatology_values_min, climatology_values_max, color='blue',alpha=0.2)
            
            plt.xlabel('Date')
            plt.ylabel(f'{var}')
            plt.title(f'{var} Time Series Comparison With Climatology')
            plt.legend()
            plt.grid(True)
            
            # Save the plot
            plot_path = os.path.join(output_dir, f'{var}/{var}_time_series_{start_date}_{end_date}_daily_shaded_area.png')
            plt.savefig(plot_path)
            plt.close()

            plt.figure(figsize=(12, 8))
            plt.plot(date_range, np.array([np.mean(arr) for arr in time_series[var]]), label=f'{var} observed', color='blue')
            plt.plot(extended_dates, climatology_values_mean, label=f'{var} climatological mean', color='red', linestyle='--')

            plt.fill_between(extended_dates, climatology_values_mean - 2*climatology_values_std, climatology_values_mean + 2*climatology_values_std, color='blue',alpha=0.2)
            
            plt.xlabel('Date')
            plt.ylabel(f'{var}')
            plt.title(f'{var} Time Series Comparison With Climatology')
            plt.legend()
            plt.grid(True)
            
            # Save the plot
            plot_path = os.path.join(output_dir, f'{var}/{var}_time_series_{start_date}_{end_date}_daily_mean_std_shaded_area.png')
            plt.savefig(plot_path)
            plt.close()
            
        if var in ['U10M_max', 'U10M_min', 'V10M_max', 'V10M_min']:
            clim_var = variable_map.get(var)
            # Se l'intervallo copre più di un anno, ripetiamo la serie climatologica per tutti gli anni necessari
            if end_year > start_year:
                extended_dates = pd.date_range(start=f'{start_year}-01-01', end=f'{end_year}-12-31', freq='D')
                extended_dates = extended_dates[~((extended_dates.month == 2) & (extended_dates.day == 29))]  # Rimuovere eventuali 29 febbraio
                climatology_values_max = np.tile(clim_series[clim_var]['max'].values, len(extended_dates) // 365)[0]
                climatology_values_min = np.tile(clim_series[clim_var]['min'].values, len(extended_dates) // 365)[0]
            else:
                extended_dates = climatology_dates
                climatology_values_max = clim_series[clim_var]['max'].values
                climatology_values_min = clim_series[clim_var]['min'].values
            if var in ['U10M_max', 'V10M_max']:
                plt.figure(figsize=(12, 8))
                plt.plot(date_range, time_series[var], label=f'{var} observed', color='blue')
                plt.plot(extended_dates, climatology_values_max, label=f'{var} climatological max', color='green', linestyle='--')
                plt.xlabel('Date')
                plt.ylabel(f'{var}')
                plt.title(f'{var} Max Time Series Comparison With Climatology')
                plt.legend()
                plt.grid(True)
            
                # Save the plot
                plot_path = os.path.join(output_dir, f'{var}/{var}_time_series_{start_date}_{end_date}.png')
                plt.savefig(plot_path)
                plt.close()
            if var in ['U10M_min', 'V10M_min']:
                plt.figure(figsize=(12, 8))
                plt.plot(date_range, time_series[var], label=f'{var} observed', color='blue')
                plt.plot(extended_dates, climatology_values_min, label=f'{var} climatological min', color='orange', linestyle='--')
                plt.xlabel('Date')
                plt.ylabel(f'{var}')
                plt.title(f'{var} Min Time Series Comparison With Climatology')
                plt.legend()
                plt.grid(True)
            
                # Save the plot
                plot_path = os.path.join(output_dir, f'{var}/{var}_time_series_{start_date}_{end_date}.png')
                plt.savefig(plot_path)
                plt.close()
        if var in ['LSM']:
            plt.figure(figsize=(12, 8))
            plt.plot(date_range_4h, np.concatenate(time_series[var]), label=f'{var} observed', color='blue')
            plt.xlabel('Date')
            plt.ylabel(f'{var} sum')
            plt.title(f'{var} Sum Time Series')
            plt.legend()
            plt.grid(True)
        
            # Save the plot
            plot_path = os.path.join(output_dir, f'{var}/{var}_sum_time_series_{start_date}_{end_date}.png')
            plt.savefig(plot_path)
            plt.close()
            

In [5]:
data_dir='/data/inputs/METOCEAN/historical/model/atmos/ECMWF/IFS_010/analysis/6h/netcdf/'
climatology_dir='/work/cmcc/mg28621/out_ECMWF_Tool_Checks/'
output_plot_dir='/work/cmcc/mg28621/out_ECMWF_Tool_Checks/output_checks/'
start_date='2024-01-01'
end_date='2024-07-31'

limite_dict = {'T2M':10,'D2M':10,'MSL':2500,'U10M':15,'V10M':15}
uom_dict = {'T2M':'K','D2M':'K/°C','MSL':'Pa','U10M':'m/s','V10M':'m/s'}

if end_date is None:
    end_date = start_date

# Load climatological statistics (precomputed)
climatology_stats = load_climatology_stats(climatology_dir)

# Itera su ciascun parametro ('T2M', 'D2M', etc.)
for param in climatology_stats:
    # Prendi la maschera LSM per il parametro corrente
    lsm_mask = climatology_stats['LSM']['mean']
    
    # Itera su ciascun tipo di statistica ('mean', 'min', 'max', etc.)
    for stat in climatology_stats[param]:
        # Applica la maschera: imposta NaN dove LSM è 1
        climatology_stats[param][stat] = climatology_stats[param][stat].where(lsm_mask != 1, np.nan)

In [ ]:
os.makedirs(output_plot_dir, exist_ok=True)

# Loop through each date in the specified range
date_range = pd.date_range(start=start_date, end=end_date)
mask = ~((date_range.month == 2) & (date_range.day == 29))
date_range = date_range[mask]

date_range_4h = pd.date_range(start=start_date, end=end_date, freq='6h')
last_day = pd.to_datetime(end_date).normalize()
last_day_hours = pd.date_range(start=last_day, end=last_day + pd.Timedelta(hours=18), freq='6h')
# Aggiungi gli orari mancanti solo se non sono già inclusi
date_range_4h = date_range_4h.union(last_day_hours)
mask = ~((date_range_4h.month == 2) & (date_range_4h.day == 29))
date_range_4h = date_range_4h[mask]

# Calculate time series
time_series, clim_series, ds_day = calculate_time_series(climatology_stats, start_date, end_date, date_range, data_dir)

# Generate and save time series plots
plot_time_series(time_series, clim_series, start_date, end_date, date_range, date_range_4h, output_plot_dir)

In [7]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature

In [ ]:
start_year = datetime.strptime(start_date, '%Y-%m-%d').year
end_year = datetime.strptime(end_date, '%Y-%m-%d').year
# Creiamo una data range per l'anno climatologico di riferimento (es. 2020)
climatology_dates = pd.date_range(start=f'{start_year}-01-01', end=f'{start_year}-12-31', freq='D')
# Rimuovere il 29 febbraio se esiste
climatology_dates = climatology_dates[~((climatology_dates.month == 2) & (climatology_dates.day == 29))]

day_time=['00','06','12','18']

for var in time_series:
    print(var)
    if var in ['T2M', 'D2M', 'MSL', 'U10M', 'V10M']:
    #if var in ['T2M']:
        # Se l'intervallo copre più di un anno, ripetiamo la serie climatologica per tutti gli anni necessari
        if end_year > start_year:
            extended_dates = pd.date_range(start=f'{start_year}-01-01', end=f'{end_year}-12-31', freq='D')
            extended_dates = extended_dates[~((extended_dates.month == 2) & (extended_dates.day == 29))]  # Rimuovere eventuali 29 febbraio
            climatology_values_mean = np.tile(clim_series[var]['mean'].values, len(extended_dates) // 365)[0]
            climatology_values_std = np.tile(clim_series[var]['std'].values, len(extended_dates) // 365)[0]
            climatology_values_max = np.tile(clim_series[var]['max'].values, len(extended_dates) // 365)[0]
            climatology_values_min = np.tile(clim_series[var]['min'].values, len(extended_dates) // 365)[0]
        else:
            extended_dates = climatology_dates
            climatology_values_std = clim_series[var]['std'].values
            climatology_values_mean = clim_series[var]['mean'].values
            climatology_values_max = clim_series[var]['max'].values
            climatology_values_min = clim_series[var]['min'].values
            
        observed_values = np.array([np.mean(arr) for arr in time_series[var]])
        upper_bound = climatology_values_mean + 2 * climatology_values_std
        lower_bound = climatology_values_mean - 2 * climatology_values_std

        # Identify indices where conditions are satisfied (range exceeded)
        exceeding_indices = np.where((observed_values > upper_bound[:len(observed_values)]) | (observed_values < lower_bound[:len(observed_values)]))[0]
        
        # Get corresponding days
        exceeding_days = date_range[exceeding_indices]
        print(f"The days that exceed the range limits for {var} are: {exceeding_days}")

        for day in exceeding_days:
            
            # Load climatological dataset for specific day
            climatology_data = climatology_stats[var]['mean'][day.dayofyear - 1]  # -1 per indice zero-based
            
            year, month, day_i = day.year, day.month, day.day
            dayofyear = day.timetuple().tm_yday
        
            file_pattern = os.path.join(data_dir, f'{year}/{month:02d}', f'*{year:02d}{month:02d}{day_i:02d}*MEDATL*.nc')
            files = sorted(glob.glob(file_pattern))  # Usa glob per trovare i file corrispondenti
        
            if not files:
                print(f"No files found for the day {day.strftime('%Y-%m-%d')}")
                continue
        
            ds = xr.open_dataset(files[0])
            ds_aligned = align_to_climatology(ds, climatology_stats)
            observation_data_tot = ds_aligned[var]
            
            daily_mean = ds_aligned[[var]].mean(dim='time')
            
            # Calculate daily max and min for U10M and V10M
            if var in ['U10M','V10M']:
                daily_max = ds_aligned[var].max(dim='time')
                daily_min = ds_aligned[var].min(dim='time')
            
            
            # Perform checks against climatology
            print("perform checks against climatology")
    
            alerts = {
                'date': day.strftime('%Y-%m-%d'),
                f'{var}_mean_alert': check_range(daily_mean[var], climatology_stats[var], dayofyear)
            }

            if var in ['U10M','V10M']:
                alerts.update({
                        f'{var}_min_alert': check_min(daily_min[var], climatology_stats[var], dayofyear),
                        f'{var}_max_alert': check_max(daily_max[var], climatology_stats[var], dayofyear)
                    })

            
            fig, ax = plt.subplots(figsize=(10, 5), subplot_kw={'projection': ccrs.PlateCarree()})
            ticks = np.linspace(min(observation_data_tot.min(),climatology_data.min()),max(observation_data_tot.max(),climatology_data.max()),5,endpoint=True)
            climatology_data.plot(ax=ax, cmap='coolwarm', transform=ccrs.PlateCarree(), vmin=min(observation_data_tot.min(),climatology_data.min()), vmax=max(observation_data_tot.max(),climatology_data.max()), cbar_kwargs={'shrink': 0.7, 'aspect': 30, 'pad': 0.15, 'ticks': ticks})
            
            plt.title(f"{var} Climatology for day {day.strftime('%Y-%m-%d')}", fontsize=20)
            
            ax.add_feature(cfeature.LAND, facecolor='grey')
            ax.coastlines()
            ax.gridlines(draw_labels=True)
            climatology_plot_path = os.path.join(output_plot_dir, f"{var}/climatology_{var}_{day.strftime('%Y%m%d')}.png")
            plt.savefig(climatology_plot_path)
            plt.close()

            norm = TwoSlopeNorm(vmin=-limite_dict[var], vcenter=0, vmax=limite_dict[var])
            ticks = np.linspace(-limite_dict[var],limite_dict[var],5,endpoint=True)
            fig, ax = plt.subplots(figsize=(10, 5), subplot_kw={'projection': ccrs.PlateCarree()})
            (daily_mean[var]-climatology_data).where(alerts[f'{var}_mean_alert']).plot(ax=ax, cmap='coolwarm', transform=ccrs.PlateCarree(),norm=norm, cbar_kwargs={'shrink': 0.7, 'aspect': 30, 'pad': 0.15, 'ticks': ticks, 'label': 'bias'})
            
            plt.title(f"{var} Differences in exceeding grid points for day {day.strftime('%Y-%m-%d')}", fontsize=14)
            
            ax.add_feature(cfeature.LAND, facecolor='grey')
            ax.coastlines()
            ax.gridlines(draw_labels=True)
            exceeding_differences_plot_path = os.path.join(output_plot_dir, f"{var}/exceeding_differences_{var}_{day.strftime('%Y%m%d')}.png")
            plt.savefig(exceeding_differences_plot_path)
            plt.close()

            fig, axs = plt.subplots(2, 2, figsize=(10, 5), subplot_kw={'projection': ccrs.PlateCarree()})
            fig.suptitle(f"{var} ECMWF data for the day {day.strftime('%Y-%m-%d')}", fontsize=15)

            norm = TwoSlopeNorm(vmin=min(observation_data_tot.min(),climatology_data.min()), vcenter= (min(observation_data_tot.min(),climatology_data.min())+max(observation_data_tot.max(),climatology_data.max()))/2, vmax=max(observation_data_tot.max(),climatology_data.max()))
            ticks = np.linspace(min(observation_data_tot.min(),climatology_data.min()),max(observation_data_tot.max(),climatology_data.max()),5,endpoint=True)
            
            for i,time_step in enumerate(range(len(ds.time))):
                # Load observations for the variable at the 4 time steps of the day
                observation_data = ds_aligned[var].isel(time=time_step)
                
                row, col = divmod(time_step, 2)
                ax = axs[row, col]
                
                observation_data.plot(ax=ax, cmap='coolwarm', transform=ccrs.PlateCarree(),norm=norm,add_colorbar=False)
                ax.set_title(f"{day_time[i]}",fontsize=10)
                
                ax.add_feature(cfeature.LAND, facecolor='grey')
                ax.coastlines()
                ax.gridlines(draw_labels=True)

            # Creare una singola colorbar in basso
            plt.subplots_adjust(hspace=0.6, wspace=0.6)
            sm = plt.cm.ScalarMappable(cmap='coolwarm', norm=norm)
            sm.set_array([])
            cbar = fig.colorbar(sm, ax=axs, orientation='horizontal', shrink=0.4, aspect=30, pad=0.1, ticks=ticks)
            cbar.set_label(f'Obs ({uom_dict[var]})')

            observation_plot_path = os.path.join(output_plot_dir, f"{var}/observations_{var}_{day.strftime('%Y%m%d')}.png")
            plt.savefig(observation_plot_path)
            plt.close()

            fig, axs = plt.subplots(2, 2, figsize=(10, 5), subplot_kw={'projection': ccrs.PlateCarree()})
            fig.suptitle(f"{var} Differences Data-Clim for the day {day.strftime('%Y-%m-%d')}",fontsize=15)

            norm = TwoSlopeNorm(vmin=-limite_dict[var], vcenter=0, vmax=limite_dict[var])
            ticks = np.linspace(-limite_dict[var],limite_dict[var],5,endpoint=True)
            
            for i,time_step in enumerate(range(len(ds.time))):
                # Load observations for the variable at the 4 time steps of the day
                observation_data = ds_aligned[var].isel(time=time_step)
                
                row, col = divmod(time_step, 2)
                ax = axs[row, col]
                
                (observation_data-climatology_data).plot(ax=ax, transform=ccrs.PlateCarree(), cmap='RdBu_r',norm=norm,add_colorbar=False)
        
                
                
                ax.set_title(f"{day_time[i]}",fontsize=10)
                
                ax.add_feature(cfeature.LAND, facecolor='grey')
                ax.coastlines()
                ax.gridlines(draw_labels=True)
            # Creare una singola colorbar in basso
            plt.subplots_adjust(hspace=0.6, wspace=0.6)
            sm = plt.cm.ScalarMappable(cmap='RdBu_r', norm=norm)
            sm.set_array([])
            cbar = fig.colorbar(sm, ax=axs, orientation='horizontal', shrink=0.4, aspect=30, pad=0.1, ticks=ticks)
            cbar.set_label(f'Differences Obs-Clim ({uom_dict[var]})')

            differences_plot_path = os.path.join(output_plot_dir, f"{var}/differences_{var}_{day.strftime('%Y%m%d')}.png")
            plt.savefig(differences_plot_path)
            plt.close()
    else:
        continue